In [7]:
# Larger LSTM Network to Generate Text for Alice in Wonderland
import numpy
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
# load ascii text and covert to lowercase
import sys

from nltk.corpus import PlaintextCorpusReader
import os
import sys
wordlists = PlaintextCorpusReader("Nepali_Corpus", '.*txt')
data = wordlists.fileids()[:10]
text = []
invalid = 0
for i in data:
    with open(os.path.join("Nepali_Corpus",i),encoding='utf8') as file:
        try:
            text.append(file.read())
        except UnicodeDecodeError :
            print(i)
            invalid+=1
print(invalid)
raw_text = "".join(text)
raw_text = raw_text.replace("\n","")

0


In [8]:

# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)
# prepare the dataset of input to output pairs encoded as integers
seq_length = 200
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = np_utils.to_categorical(X)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)
# define the LSTM model


[' ', '(', ')', ',', '.', '?', '\xa0', '¥', 'ँ', 'ं', 'ः', 'अ', 'आ', 'इ', 'ई', 'उ', 'ए', 'ऐ', 'ओ', 'औ', 'क', 'ख', 'ग', 'घ', 'ङ', 'च', 'छ', 'ज', 'झ', 'ञ', 'ट', 'ठ', 'ड', 'ढ', 'ण', 'त', 'थ', 'द', 'ध', 'न', 'प', 'फ', 'ब', 'भ', 'म', 'य', 'र', 'ल', 'व', 'श', 'ष', 'स', 'ह', 'ा', 'ि', 'ी', 'ु', 'ू', 'ृ', 'े', 'ै', 'ो', 'ौ', '्', '।', '०', '१', '२', '३', '४', '५', '६', '७', '८', '९', '–', '‘', '’']
Total Characters:  13424
Total Vocab:  78
Total Patterns:  13224


In [9]:
X.shape

(13224, 200, 78)

In [10]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(X, y, epochs=50, batch_size=64, callbacks=callbacks_list)

Epoch 1/50
 5440/13224 [===========>..................] - ETA: 5:34 - loss: 3.5775

KeyboardInterrupt: 

In [ ]:
model.save("2-layer-200-weights-one-hot.h5")

In [5]:
model = keras.models.load_model("2-layer-20-weights-one-hot.h5")

In [6]:
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Seed:")
print (''.join([int_to_char[value] for value in pattern]), "\n")
print("".join([int_to_char[value] for value in pattern]),end="")
# generate characters
f = open("sent_dump","w")
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), -1))
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    print(result,sep='',end='')
    f.write("".join([int_to_char[int(j*n_vocab)] 
            for i in x[0]
               for j in i
           ])+"-->"+result+"\n")
    pattern.append(np_utils.to_categorical(index))
    pattern = pattern[1:len(pattern)]
print ("\nDone.")
f.close()

Seed:
 ‘लेखककी स्वास्नी’ उपन्यास प्रकाशित छन् । पुस्तक ‘इण्डिगो’ प्रकाशनले बजारमा ल्याएको हो । काठमाडौ,– चिनियाँ कथाकार लु स्युनको कथामा आधारित ‘म्याड म्यान्स डायरी’को मञ्चनसँगै  राजधानीमा नयाँ नाटघकर खुलेक 

 ‘लेखककी स्वास्नी’ उपन्यास प्रकाशित छन् । पुस्तक ‘इण्डिगो’ प्रकाशनले बजारमा ल्याएको हो । काठमाडौ,– चिनियाँ कथाकार लु स्युनको कथामा आधारित ‘म्याड म्यान्स डायरी’को मञ्चनसँगै  राजधानीमा नयाँ नाटघकर खुलेक

ValueError: Error when checking : expected lstm_3_input to have shape (20, 1) but got array with shape (200, 1)